In [1]:
import pandas as pd
import quandl # dataset providing package, now iam using kaggle for downloading sample data
import math
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [28]:
df = pd.read_csv('data_files/WIKI_PRICES.csv')
df.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,31.041951,34.112034,27.289627,30.018590,44739900.0
1,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,29.295415,29.336350,27.160002,27.548879,10897100.0
2,A,1999-11-22,41.31,44.00,40.06,44.00,4705200.0,0.0,1.0,28.183363,30.018590,27.330562,30.018590,4705200.0
3,A,1999-11-23,42.50,43.63,40.25,40.25,4274400.0,0.0,1.0,28.995229,29.766161,27.460188,27.460188,4274400.0
4,A,1999-11-24,40.13,41.94,40.00,41.06,3464400.0,0.0,1.0,27.378319,28.613174,27.289627,28.012803,3464400.0


In [3]:
df = df[['adj_open','adj_high','adj_low','adj_close','adj_volume']]

In [4]:
# high low percentage
df['HL_PCT'] = (df['adj_high'] - df['adj_close']) / df['adj_close'] * 100.0

In [5]:
# percentage change new - old / old * 100.0
df['PCT_change'] = (df['adj_close'] - df['adj_open']) / df['adj_open'] * 100.0

In [6]:
df = df[['adj_close','HL_PCT','PCT_change','adj_volume']]

In [7]:
df.head()

,adj_close,HL_PCT,PCT_change,adj_volume
0,30.018590,13.636364,-3.296703,44739900.0
1,27.548879,6.488361,-5.961807,10897100.0
2,30.018590,0.000000,6.511740,4705200.0
3,27.460188,8.397516,-5.294118,4274400.0
4,28.012803,2.143205,2.317468,3464400.0


In [8]:
forecast_col = 'adj_close'

In [9]:
df.fillna(-99999, inplace=True)

In [11]:
forecast_out = int(math.ceil(0.1 * len(df))) # 10 percentage data
print(forecast_out)

1538932


In [12]:
df['label'] = df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)

In [13]:
df.head()

,adj_close,HL_PCT,PCT_change,adj_volume,label
0,30.018590,13.636364,-3.296703,44739900.0,27.130393
1,27.548879,6.488361,-5.961807,10897100.0,27.400187
2,30.018590,0.000000,6.511740,4705200.0,26.956626
3,27.460188,8.397516,-5.294118,4274400.0,23.380701
4,28.012803,2.143205,2.317468,3464400.0,22.553025


In [14]:
df.tail()

,adj_close,HL_PCT,PCT_change,adj_volume,label
13850377,14.640,0.717213,0.273973,456800.0,23.95
13850378,14.750,0.000000,0.067843,771200.0,23.35
13850379,14.975,2.003339,1.525424,873200.0,23.55
13850380,14.730,2.647658,-1.438608,488400.0,24.65
13850381,14.605,2.636084,-1.517195,427000.0,23.60


In [15]:
X = np.array(df.drop(columns=['label']))  # Features
y = np.array(df['label'])

X = preprocessing.scale(X)

In [16]:
# X = X[:-forcast_out+1]
df.dropna(inplace=True)
y = np.array(df['label'])

In [17]:
print(len(X), len(y))

13850382 13850382


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
clf = LinearRegression()

In [20]:
clf.fit(X_train, y_train)

LinearRegression()

In [21]:
accuracy = clf.score(X_test, y_test)

In [22]:
accuracy # less accuracy

0.0006125025994003774

In [24]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score

# 2. Time Series Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)  # Adjust n_splits as needed

rmse_scores = []
r2_scores = []

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 3. Feature Scaling (Important: Scale WITHIN each fold)
    scaler = preprocessing.StandardScaler() # Initialize the Scaler

    X_train = scaler.fit_transform(X_train) # Fit and transform training data
    X_test = scaler.transform(X_test) # Transform test data using the fitted scaler

In [25]:
clf.fit(X_train, y_train)

# 5. Prediction and Evaluation
y_pred = clf.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred) # Use r2_score for R-squared

rmse_scores.append(rmse)
r2_scores.append(r2)

print(f"Fold: {len(rmse_scores)}, RMSE: {rmse}, R-squared: {r2}")

# 6. Overall Performance
avg_rmse = np.mean(rmse_scores)
avg_r2 = np.mean(r2_scores)

print(f"\nAverage RMSE: {avg_rmse}")
print(f"Average R-squared: {avg_r2}")

Fold: 1, RMSE: 9917.635840156727, R-squared: -0.0015622666822932008

Average RMSE: 9917.635840156727
Average R-squared: -0.0015622666822932008


In [26]:
# performing badly

In [31]:
df = df[['open', 'high', 'low', 'close', 'volume']]
df['HL_PCT'] = (df['high'] - df['low']) / df['low'] * 100.0
df['PCT_change'] = (df['close'] - df['open']) / df['open'] * 100.0

df = df[['close', 'HL_PCT', 'PCT_change', 'volume']]

forecast_col = 'close'
df.fillna(-9999, inplace=True)
forecast_out = int(math.ceil(0.01 * len(df)))
df['label'] = df[forecast_col].shift(-forecast_out)

X = np.array(df.drop(['label'], axis=1))
X = preprocessing.scale(X)
X_lately = X[-forecast_out:]
X = X[:-forecast_out:]

df.dropna(inplace=True)
y = np.array(df['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = LinearRegression(n_jobs=-1)
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)

/tmp/ipykernel_10991/3260954031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HL_PCT'] = (df['high'] - df['low']) / df['low'] * 100.0


In [33]:
print(accuracy) # performing badly

2.925321332503117e-06
